**Set environment**

In [1]:
suppressMessages(suppressWarnings(source("../config/config_sing.R")))
show_env()

You are in Singularity: singularity_proj_encode_fcc 
BASE DIRECTORY (FD_BASE): /data/reddylab/Kuei 
WORK DIRECTORY (FD_WORK): /data/reddylab/Kuei/out 
CODE DIRECTORY (FD_CODE): /data/reddylab/Kuei/code 
PATH OF PROJECT (FD_PRJ): /data/reddylab/Kuei/code/Proj_CombEffect_ENCODE_FCC 
PATH OF RESULTS (FD_RES): /data/reddylab/Kuei/out/proj_combeffect_encode_fcc 
PATH OF LOG     (FD_LOG): /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/log 


**Check data**

In [5]:
fdiry = file.path(
    FD_RES, 
    "results", 
    "region", 
    "KS91_K562_ASTARRseq_peak_macs_input", 
    "annotation_chromHMM")
dir(fdiry)

[1] "peak.annotation.ChIP_ENCSR000EWA-ENCSR000AKP-ENCSR000EWC-ENCSR000DWB-ENCSR000EWB-ENCSR000APE.bed.gz"

In [6]:
fdiry = file.path(
    FD_RES, 
    "results", 
    "region", 
    "annotation_chromHMM")
dir(fdiry)

[1] "ChIP_ENCSR000EWA-ENCSR000AKP-ENCSR000EWC-ENCSR000DWB-ENCSR000EWB-ENCSR000APE.bed"
[2] "description.tsv"

## Import data

In [7]:
fdiry = file.path(
    FD_RES, 
    "results", 
    "region", 
    "annotation_chromHMM")
fname = "description.tsv"
fpath = file.path(fdiry, fname)

dat = read_tsv(fpath)

dat_cnames = dat
print(dim(dat))
dat

Rows: 9 Columns: 2
── Column specification ─────────────────────────────────────────────────────────────────────
Delimiter: "\t"
chr (2): Name, Description

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


[1] 9 2


Name,Description
<chr>,<chr>
Chrom,Chromosome
Start,Start position
End,End position
Name,ChromHMM Label
Score,Score
Strand,[+-.]; Use '.' if no strand is assigned.
ThickStart,The starting position at which the feature is drawn thickly
ThickEnd,The ending position at which the feature is drawn thickly
ItemRgb,"An RGB value of the form R,G,B (e.g. 255,0,0)"


In [9]:
### init
annotation = "ChromHMM"
cnames = dat_cnames$Name
cnames = c("Chrom_ATAC", "Start_ATAC", "End_ATAC", cnames, "Overlap")

###
fdiry = file.path(
    FD_RES, 
    "results", 
    "region", 
    "KS91_K562_ASTARRseq_peak_macs_input", 
    "annotation_chromHMM")
fname = "peak.annotation.ChIP_ENCSR000EWA-ENCSR000AKP-ENCSR000EWC-ENCSR000DWB-ENCSR000EWB-ENCSR000APE.bed.gz"
fpath = file.path(fdiry, fname)

### read data
dat = read_tsv(fpath, col_names = cnames, show_col_types = FALSE)
dat = dat %>% dplyr::mutate(
    Peak_ATAC  = paste(Chrom_ATAC, Start_ATAC, End_ATAC, sep="_"),
    Annotation = annotation,
    Label      = Name
)

### assign and show
dat_peak_annot = dat
print(dim(dat))
head(dat)

[1] 375968     16


Chrom_ATAC,Start_ATAC,End_ATAC,Chrom,Start,End,Name,Score,Strand,ThickStart,ThickEnd,ItemRgb,Overlap,Peak_ATAC,Annotation,Label
<chr>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<chr>,<dbl>,<chr>,<dbl>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<chr>
chr1,10015,10442,chr1,0,16000,Quies,1,.,0,16000,"220,220,220",427,chr1_10015_10442,ChromHMM,Quies
chr1,14253,14645,chr1,0,16000,Quies,1,.,0,16000,"220,220,220",392,chr1_14253_14645,ChromHMM,Quies
chr1,16015,16477,chr1,16000,16200,TxWk,1,.,16000,16200,"63,154,80",185,chr1_16015_16477,ChromHMM,TxWk
chr1,16015,16477,chr1,16200,17400,Quies,1,.,16200,17400,"220,220,220",277,chr1_16015_16477,ChromHMM,Quies
chr1,17237,17772,chr1,17400,17600,TxWk,1,.,17400,17600,"63,154,80",200,chr1_17237_17772,ChromHMM,TxWk
chr1,17237,17772,chr1,16200,17400,Quies,1,.,16200,17400,"220,220,220",163,chr1_17237_17772,ChromHMM,Quies


## Arrange and summarize

In [10]:
dat = dat_peak_annot
dat = dat %>% 
    dplyr::group_by(Peak_ATAC, Annotation, Label) %>%
    dplyr::summarise(Count = n(), .groups = "drop")

dat = dat %>% dplyr::rename("Peak" = "Peak_ATAC")

### assign and show
dat_peak_annot_count = dat
print(dim(dat))
head(dat)

[1] 343446      4


Peak,Annotation,Label,Count
<chr>,<chr>,<chr>,<int>
chr10_100009096_100010466,ChromHMM,TssFlnk,3
chr10_100009096_100010466,ChromHMM,TssFlnkD,2
chr10_100020710_100021134,ChromHMM,Biv,1
chr10_100020710_100021134,ChromHMM,ReprPC,1
chr10_1000372_1000820,ChromHMM,Tx,1
chr10_100045961_100046653,ChromHMM,Quies,1


In [11]:
dat = dat_peak_annot_count
table(dat$Count)


     1      2      3      4      5      6 
313666  27329   2193    231     21      6 

## Save results

In [12]:
fdiry = file.path(FD_RES, "results", "region", "KS91_K562_ASTARRseq_peak_macs_input", "summary")
fname = "peak.summary.chromHMM.tsv"
fpath = file.path(fdiry, fname)

dat = dat_peak_annot_count
write_tsv(dat, fpath)